In [37]:
#Import necessary libraries
import numpy as np
import pandas as pd
import textwrap
import math 
import time

In [38]:
#Load DataSet
start_time = time.time() 
time_tracker=0
x = pd.read_table('HW5_dataset.txt',delim_whitespace = True,header=0)
x_training = x[0:140][:]  #Taking first 140 samples into training data
x_test = x[140:][:]       #Taking remaining samples into test data
x_training = x_training.values
x_test = x_test.values
x = x.values
time_tracker += time.time() - start_time

In [39]:
#Separate data
start_time = time.time()
species_zero_training = x_training[[x_training[:,0]==0]][:]
species_one_training  = x_training[[x_training[:,0]==1]][:]
species_zero_training = species_zero_training[:,1:]
species_one_training = species_one_training[:,1:]

species_zero_test = x_test[[x_test[:,0]==0]][:]
species_one_test =  x_test[[x_test[:,0]==1]][:]
species_zero_test = species_zero_test[:,1:]
species_one_test = species_one_test[:,1:]

species_zero = x[[x[:,0]==0]][:]
species_one = x[[x[:,0]==1]][:]
species_zero= species_zero[:,1:]
species_one = species_one[:,1:]
print(len(species_zero))
print((len(species_one)))
time_tracker += time.time() - start_time

100
100


In [40]:
start_time = time.time()
def mean(data):
    mean = np.zeros((1,7))
    for i in range(0,len(data)):
        mean += data[i][:]
    mean = mean/len(data)
    return mean
def covariance(data,mean):
    covariance = np.zeros((7,7))
    for i in range(0,len(data)):
        a = (data[i,:] - mean)
        covariance += (a.T@a) 
    covariance = (1/len(data))*covariance
    return covariance
time_tracker += time.time() - start_time

In [41]:
start_time = time.time()
#mean_one = mean(species_one)
#mean_zero = mean(species_zero)
mean_zero_training = mean(species_zero_training)
mean_one_training = mean(species_one_training)
#mean_zero_test = mean(species_zero_test)
#mean_one_test = mean(species_one_test)
#covariance_zero = covariance(species_zero,mean_zero)
#covariance_one = covariance(species_one,mean_one)
covariance_zero_training = covariance(species_zero_training,mean_zero_training)
#covariance_zero_test = covariance(species_zero_test,mean_zero_test)
covariance_one_training = covariance(species_one_training,mean_one_training)
#covariance_one_test = covariance(species_one_test,mean_one_test)
#covariance_one = covariance_one + 0.01*np.identity(7)
#covariance_one_test = covariance_one_test + 0.0001*np.identity(7)
#covariance_zero = covariance_zero +0.01*np.identity(7)
print(covariance_zero_training)
print(covariance_one_training)
time_tracker += time.time() - start_time

[[  9.1691358    7.13141204  18.57479167  20.85845679   8.56783179
   -0.42507716   0.42507716]
 [  7.13141204   6.76387153  14.15255208  16.34744213   6.4666956
   -0.78153935   0.78153935]
 [ 18.57479167  14.15255208  38.45012153  42.91434028  17.76977431
   -0.62760417   0.62760417]
 [ 20.85845679  16.34744213  42.91434028  48.30138117  19.79787423
   -0.91253858   0.91253858]
 [  8.56783179   6.4666956   17.76977431  19.79787423   8.32226659
   -0.25854552   0.25854552]
 [ -0.42507716  -0.78153935  -0.62760417  -0.91253858  -0.25854552
    0.2498071   -0.2498071 ]
 [  0.42507716   0.78153935   0.62760417   0.91253858   0.25854552
   -0.2498071    0.2498071 ]]
[[  9.08346021   5.88378028  20.87943772  23.8567474    9.32790657
    0.55536332  -0.55536332]
 [  5.88378028   4.50396194  13.5232699   15.55649654   6.15652249
    0.06842561  -0.06842561]
 [ 20.87943772  13.5232699   48.49261894  55.29956531  21.56766003
    1.3803417   -1.3803417 ]
 [ 23.8567474   15.55649654  55.29956531

In [42]:
start_time = time.time()
#covariance_one_inverse = np.linalg.inv(covariance_one)
#covariance_zero_inverse = np.linalg.inv(covariance_zero)
covariance_one_training_inverse = np.linalg.inv(covariance_one_training)
#covariance_one_test_inverse = np.linalg.inv(covariance_one_test)
covariance_zero_training_inverse = np.linalg.inv(covariance_zero_training)
#covariance_zero_test_inverse = np.linalg.inv(covariance_zero_test)
time_tracker += time.time() - start_time

In [43]:
#Take prior for whole data
start_time = time.time()
p0 = len(species_zero)/len(x)
p1 = len(species_one)/len(x)

#Take prior for training data 
p0_training = len(species_zero_training)/len(x_training)
p1_training = len(species_one_training)/len(x_training)

#Take prior for test data
p0_test = len(species_zero_test)/len(x_test)
p1_test = len(species_one_test)/len(x_test)
time_tracker += time.time() - start_time

In [44]:
start_time = time.time()
def score_class0_training(x):
    return (-1/2)*(((x-mean_zero_training))@covariance_zero_training_inverse@(x-mean_zero_training).T) 
         -3.5*(math.log(2*math.pi)) - 0.5*math.log(abs(np.linalg.det(covariance_zero_training)))+math.log(p0_training)

def score_class1_training(x):
    return (-1/2)*(((x-mean_one_training))@covariance_one_training_inverse@(x-mean_one_training).T) 
        -3.5*(math.log(2*math.pi)) - 0.5*math.log(abs(np.linalg.det(covariance_zero_training)))+math.log(p1_training)
zeros_training=[]
ones_training=[]
for i in range(0,len(x_training)):
    score_0_training = score_class0_training(x_training[i][1:])
    score_1_training = score_class1_training(x_training[i][1:])
    if(score_0_training>score_1_training):
        zeros_training.append(i)
    else:
        ones_training.append(i)
print('Number of class 0 Species: ' +str(len(zeros_training)))
print('Number of class 1 Species: ' +str(len(ones_training)))        
time_tracker += time.time() - start_time  

Number of class 0 Species: 72
Number of class 1 Species: 68


In [45]:
start_time = time.time()
def score_class0(x):
    return (-1/2)*(((x-mean_zero))@covariance_zero_inverse@(x-mean_zero).T) -3.5*(math.log(2*math.pi)) - 0.5*math.log(np.linalg.det(covariance_zero))+math.log(p0)

def score_class1(x):
    return (-1/2)*(((x-mean_one))@covariance_one_inverse@(x-mean_one).T) -3.5*(math.log(2*math.pi)) - 0.5*math.log(np.linalg.det(covariance_zero))+math.log(p1)

zeros=[]
ones=[]
scores0 =[]
scores1 =[]
for i in range(0,len(x)):
    score_0 = score_class0_training(x[i][1:])
    score_1 = score_class1_training(x[i][1:])
    scores0.append(score_0)
    scores1.append(score_1)
    if(score_0>score_1):
        zeros.append(i)
    else:
        ones.append(i)
print('Number of class 0 Species: ' +str(len(zeros)))
print('Number of class 1 Species: ' +str(len(ones)))
time_tracker += time.time() - start_time

Number of class 0 Species: 100
Number of class 1 Species: 100


In [46]:
start_time = time.time()
def score_class0_test(x):
    return (-1/2)*(((x-mean_zero_test))@covariance_zero_test_inverse@(x-mean_zero_test).T) -3.5*(math.log(2*math.pi)) - 0.5*math.log(np.linalg.det(covariance_zero_test))+math.log(p0_test)

def score_class1_test(x):
    return (-1/2)*(((x-mean_one_test))@covariance_one_test_inverse@(x-mean_one_test).T) -3.5*(math.log(2*math.pi)) - 0.5*math.log(np.linalg.det(covariance_zero_test))+math.log(p1_test)
zeros_test=[]
ones_test=[]
for i in range(0,len(x_test)):
    score_0_test = score_class0_training(x_test[i][1:])
    score_1_test = score_class1_training(x_test[i][1:])
    if(score_0_test>score_1_test):
        zeros_test.append(i)
    else:
        ones_test.append(i)
print('Number of class 0 Species: ' +str(len(zeros_test)))
print('Number of class 1 Species: ' +str(len(ones_test)))        
time_tracker += time.time() - start_time        

Number of class 0 Species: 28
Number of class 1 Species: 32


In [47]:
start_time = time.time()
true_zeros = 0
for i in range(0,len(zeros)):
    if x[zeros[i]][0]==0:
        true_zeros+=1
true_ones = 0
for i in range(0,len(ones)):
    if x[ones[i]][0]==1:
        true_ones+=1
false_zeros = len(zeros)-true_zeros
false_ones = len(ones) - true_ones

print('True Zeros = '+str(true_zeros))
print('True Ones = '+str(true_ones))
print('False Zeros = '+str(false_zeros))
print('False Ones = '+str(false_zeros))
time_tracker += time.time() - start_time

True Zeros = 100
True Ones = 100
False Zeros = 0
False Ones = 0


In [48]:
start_time = time.time()
true_zeros_training = 0
for i in range(0,len(zeros_training)):
    if x_training[zeros_training[i]][0]==0:
        true_zeros_training+=1
true_ones_training = 0
for i in range(0,len(ones_training)):
    if x_training[ones_training[i]][0]==1:
        true_ones_training+=1
false_zeros_training = len(zeros_training)-true_zeros_training
false_ones_training = len(ones_training) - true_ones_training

print('True Zeros = '+str(true_zeros_training))
print('True Ones = '+str(true_ones_training))
print('False Zeros = '+str(false_zeros_training))
print('False Ones = '+str(false_zeros_training))
time_tracker += time.time() - start_time

True Zeros = 72
True Ones = 68
False Zeros = 0
False Ones = 0


In [51]:
start_time = time.time()
true_zeros_test = 0
for i in range(0,len(zeros_test)):
    if x_test[zeros_test[i]][0]==0:
        true_zeros_test+=1
true_ones_test = 0
for i in range(0,len(ones_test)):
    if x_test[ones_test[i]][0]==1:
        true_ones_test+=1
false_zeros_test = len(zeros_test)-true_zeros_test
false_ones_test = len(ones_test) - true_ones_test
print('True Zeros = '+str(true_zeros_test))
print('True Ones = '+str(true_ones_test))
print('False Zeros = '+str(false_zeros_test))
print('False Ones = '+str(false_zeros_test))
time_tracker += time.time() - start_time

True Zeros = 28
True Ones = 32
False Zeros = 0
False Ones = 0


In [50]:
print('Time taken to finish the classification is '+ str(time_tracker)+' seconds')

Time taken to finish the classification is 0.08155059814453125 seconds


In [31]:
print(min(scores0), max(scores0))
print(min(scores1), max(scores1))


[[-59.57006186]] [[-7.08400667]]
[[-79.72610518]] [[-4.8077006]]
